In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [7]:
spark = SparkSession.builder \
    .appName("TestReadBronze") \
    .getOrCreate()

In [8]:
bronze_tables = spark.sql("SHOW TABLES IN datalake.bronze").collect()
if not bronze_tables:
    print("No bronze tables found in namespace 'datalake.bronze'.")

In [5]:
bronze_tables

[Row(namespace='bronze', tableName='Gold', isTemporary=False),
 Row(namespace='bronze', tableName='Oil', isTemporary=False),
 Row(namespace='bronze', tableName='S_P500', isTemporary=False),
 Row(namespace='bronze', tableName='USD_VND', isTemporary=False),
 Row(namespace='bronze', tableName='US_10_Year_Bond', isTemporary=False),
 Row(namespace='bronze', tableName='US_2_Year_Bond', isTemporary=False),
 Row(namespace='bronze', tableName='US_3_Month_Bond', isTemporary=False),
 Row(namespace='bronze', tableName='US_5_Year_Bond', isTemporary=False),
 Row(namespace='bronze', tableName='US_Dollar', isTemporary=False),
 Row(namespace='bronze', tableName='VND_USD', isTemporary=False),
 Row(namespace='bronze', tableName='VN_10_Year_Bond', isTemporary=False),
 Row(namespace='bronze', tableName='VN_2_Year_Bond', isTemporary=False),
 Row(namespace='bronze', tableName='VN_5_Year_Bond', isTemporary=False)]

In [9]:
gold_tables = spark.sql("SELECT * FROM datalake.bronze.Gold").collect()

In [10]:
gold_tables

[Row(Date='12/30/1993', Price='391.9', Open='393.3', High='393.9', Low='391.0', Vol='29.64K', Change='0.03%'),
 Row(Date='12/29/1993', Price='391.8', Open='386.6', High='392.5', Low='384.8', Vol='18.71K', Change='0.85%'),
 Row(Date='12/28/1993', Price='388.5', Open='389.7', High='390.0', Low='387.9', Vol='39.20K', Change='-0.15%'),
 Row(Date='12/27/1993', Price='389.1', Open='388.1', High='389.4', Low='387.7', Vol='8.38K', Change='0.54%'),
 Row(Date='12/23/1993', Price='387.0', Open='387.0', High='387.3', Low='385.6', Vol='6.91K', Change='-0.05%'),
 Row(Date='12/22/1993', Price='387.2', Open='387.3', High='388.2', Low='386.0', Vol='8.37K', Change='-0.54%'),
 Row(Date='12/21/1993', Price='389.3', Open='390.4', High='390.5', Low='388.5', Vol='15.93K', Change='-0.38%'),
 Row(Date='12/20/1993', Price='390.8', Open='391.1', High='392.2', Low='390.6', Vol='13.58K', Change='0.49%'),
 Row(Date='12/17/1993', Price='388.9', Open='387.3', High='389.1', Low='386.8', Vol='16.35K', Change='0.36%'),


In [5]:
spark.stop()